### FirstName LastName

<img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-014g-1000x734.jpg" width=140> <img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-017f-563x750.jpg" width=77> <img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-018a-605x750.jpg" width=83> <img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-021c-605x750.jpg" width=84> <img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-019-1000x689.jpg" width=150> <img src="http://interface-experience.org/site/wp-content/uploads/2015/01/IE-032-621x750.jpg" width=87>
## COM110 Programming Assignment 5:
### Modding Video Games
<img src="https://api.arcade.academy/en/latest/_images/tetris1.png" width=200>
(c) 2023 Timothy Becker, Department of Computer Science <br><br>
<img src="https://www.conncoll.edu/media/website-media/visualidentity/images/1Line-LogoSig-Color.jpg" width="200">

### Preliminary
Here you will be working with the arcade python video game library that facilitates several common video game workflows. We explored a platformer game in class to work on animation game logic and now will explore a more classic video game: Tetris. Here the game is mostly built for you and you will be modding it: which is changing the rules, and other in game variables so that the difficulty and other properties will be altered.

In [1]:
#%conda install arcade
#%pip install arcade #alterate way to install

### [Part 1 20pts] Run the game and play it to get a feel for logic and controls
Write down a sketch of how the internal logic would work if you were going to write this from scratch. Then carefully read the code to understand the key parts of the game code and answer the questions by fulfilling each question request by perturbing the original state of the game. Consider this part a series of experiments where you will learn how this game was written by play it and changing variable values to find how your actions effect gameplay.

In [ ]:
""" Tetris clone, with some ideas from silvasur's code:
https://gist.github.com/silvasur/565419/d9de6a84e7da000797ac681976442073045c74a4
this was simplified by TJB 2023 for COM110 """

import arcade
import random
import numpy as np
import PIL

# Set how many rows and columns we will have
ROW_COUNT = 24
COLUMN_COUNT = 12

# This sets the WIDTH and HEIGHT of each grid location
WIDTH = 30
HEIGHT = 30

# This sets the margin between each cell
# and on the edges of the screen.
MARGIN = 5

# Do the math to figure out our screen dimensions
SCREEN_WIDTH = (WIDTH + MARGIN) * COLUMN_COUNT + MARGIN
SCREEN_HEIGHT = (HEIGHT + MARGIN) * ROW_COUNT + MARGIN
SCREEN_TITLE = "Tetris"

colors = [
    (0,   0,   0, 255),
    (255, 0,   0, 255),
    (0,   150, 0, 255),
    (0,   0,   255, 255),
    (255, 120, 0, 255),
    (255, 255, 0, 255),
    (180, 0,   255, 255),
    (0,   220, 220, 255)
]

# Define the shapes of the single parts
tetris_shapes = [
    [[1, 1, 1],
     [0, 1, 0]],

    [[0, 2, 2],
     [2, 2, 0]],

    [[3, 3, 0],
     [0, 3, 3]],

    [[4, 0, 0],
     [4, 4, 4]],

    [[0, 0, 5],
     [5, 5, 5]],

    [[6, 6, 6, 6]],

    [[7, 7],
     [7, 7]]
]
tetris_shapes = [np.asarray(shape) for shape in tetris_shapes]

def create_textures():
    """ Create a list of images for sprites based on the global colors. """
    new_textures = []
    for color in colors:
        image = PIL.Image.new('RGBA', (WIDTH, HEIGHT), color)
        new_textures.append(arcade.Texture(str(color), image=image))
    return new_textures

texture_list = create_textures()

def rotate_counterclockwise(shape):
    """ Rotates a matrix couterclockwise """
    return shape[::-1].T

def check_collision(board, shape, offset):
    """See if the matrix stored in the shape will intersect anything
    on the board based on the offset. Offset is an (x, y) coordinate."""
    off_x, off_y = offset
    for i in range(len(shape)):
        for j in range(len(shape[i])):
            if shape[i][j]>0 and board[i+off_y][j+off_x]:
                return True
    return False

def join_matrixes(matrix_1, matrix_2, matrix_2_offset):
    """ Copy matrix 2 onto matrix 1 based on the passed in x, y offset coordinate """
    off_x, off_y = matrix_2_offset
    for i in range(len(matrix_2)):
        for j in range(len(matrix_2[i])):
            matrix_1[i+off_y-1][j+off_x] += matrix_2[i][j]
    return matrix_1


def new_board():
    """ Create a grid of 0's. Add 1's to the bottom for easier collision detection. """
    board = [[0 for _x in range(COLUMN_COUNT)] for _y in range(ROW_COUNT)]
    board += [[1 for _x in range(COLUMN_COUNT)]]
    board = np.asarray(board)
    return board

class MyGame(arcade.Window):
    def __init__(self, width, height, title):
        super().__init__(width, height, title)
        arcade.set_background_color(arcade.color.WHITE)

        self.board = None
        self.frame_count = 0
        self.game_over = False
        self.paused = False
        self.board_sprite_list = None

        self.stone = None
        self.stone_x = 0
        self.stone_y = 0

    def new_stone(self):
        """Randomly grab a new stone and set the stone location to the top.
        If we immediately collide, then game-over."""
        self.stone = random.choice(tetris_shapes)
        self.stone_x = int(COLUMN_COUNT / 2 - len(self.stone[0]) / 2)
        self.stone_y = 0

        if check_collision(self.board, self.stone, (self.stone_x, self.stone_y)):
            self.game_over = True

    def setup(self):
        self.board = new_board()

        self.board_sprite_list = arcade.SpriteList()
        for row in range(len(self.board)):
            for column in range(len(self.board[0])):
                sprite = arcade.Sprite()
                for texture in texture_list:
                    sprite.append_texture(texture)
                sprite.set_texture(0)
                sprite.center_x = (MARGIN + WIDTH) * column + MARGIN + WIDTH // 2
                sprite.center_y = SCREEN_HEIGHT - (MARGIN + HEIGHT) * row + MARGIN + HEIGHT // 2

                self.board_sprite_list.append(sprite)

        self.new_stone()
        self.update_board()

    def drop(self):
        """Drop the stone down one place. Check for collision. If collided, then
           join matrixes, Check for rows we can remove, Update sprite list with stones, Create a new stone"""
        if not self.game_over and not self.paused:
            self.stone_y += 1
            if check_collision(self.board, self.stone, (self.stone_x, self.stone_y)):
                self.board = join_matrixes(self.board, self.stone, (self.stone_x, self.stone_y))
                for i in range(len(self.board)-1):     #check every row except the bottom
                    if np.prod(self.board[i,:]) > 0:   #any 0 will make the product zero
                        self.board[0:i+1,:] = np.roll(self.board[0:i+1,:],axis=0,shift=1) #shift all rows down by one
                        self.board[0,:] = 0                                               #clear the top row that was filled
                self.update_board()
                self.new_stone()

    def rotate_stone(self):
        """ Rotate the stone, check collision. """
        if not self.game_over and not self.paused:
            new_stone = rotate_counterclockwise(self.stone)
            if self.stone_x + len(new_stone[0]) >= COLUMN_COUNT:
                self.stone_x = COLUMN_COUNT - len(new_stone[0])
            if not check_collision(self.board, new_stone, (self.stone_x, self.stone_y)):
                self.stone = new_stone

    def on_update(self, dt):
        """ Update, drop stone if warrented """
        self.frame_count += 1
        if self.frame_count % 60 == 0:
            self.drop()

    def move(self, delta_x):
        """ Move the stone back and forth based on delta x. """
        if not self.game_over and not self.paused:
            new_x = self.stone_x + delta_x
            if new_x < 0:
                new_x = 0
            if new_x > COLUMN_COUNT - len(self.stone[0]):
                new_x = COLUMN_COUNT - len(self.stone[0])
            if not check_collision(self.board, self.stone, (new_x, self.stone_y)):
                self.stone_x = new_x

    def on_key_press(self, key, modifiers):
        if key == arcade.key.LEFT:  self.move(-1)
        if key == arcade.key.RIGHT: self.move(1)
        if key == arcade.key.UP:    self.rotate_stone()
        if key == arcade.key.DOWN:  self.drop()

    # noinspection PyMethodMayBeStatic
    def draw_grid(self, grid, offset_x, offset_y):
        """Draw the grid. Used to draw the falling stones. The board is drawn
        by the sprite list."""
        for row in range(len(grid)):
            for column in range(len(grid[0])):
                if grid[row][column]:
                    color = colors[grid[row][column]]
                    x = (MARGIN + WIDTH) * (column + offset_x) + MARGIN + WIDTH // 2
                    y = SCREEN_HEIGHT - (MARGIN + HEIGHT) * (row + offset_y) + MARGIN + HEIGHT // 2
                    arcade.draw_rectangle_filled(x, y, WIDTH, HEIGHT, color)

    def update_board(self):
        """Update the sprite list to reflect the contents of the 2d grid"""
        for row in range(len(self.board)):
            for column in range(len(self.board[0])):
                v = self.board[row][column]
                i = row * COLUMN_COUNT + column
                self.board_sprite_list[i].set_texture(v)

    def on_draw(self):
        """ Render the screen. """
        self.clear()
        self.board_sprite_list.draw()
        self.draw_grid(self.stone, self.stone_x, self.stone_y)

def main():
    my_game = MyGame(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    my_game.setup()
    arcade.run()

if __name__ == "__main__":
    main()

### [Exercise 1a] Modify the on_update method
Change the value that mods the frame counter from 60 to 10 and then to 120. Then make a new variable in caps at the very top near ROW_COUNT = ... call FRAME_DIV and set it to a good starting level value based on your play experiments. Detail some of your experiments below (a few sentences about what you found out)

your markdown answer here for [Exercise 1a]

### [Exercise 1b] Modify the ROW_COUNT = 24 and COLUMN_COUNT = 12
Now change the number of rows and columns in the game and detail in your words the effects of large versus smaller rows and columns. What is the minim size to make this game playable?

your markdown answer here for [Exercise 1b]

### [Exercise 1c] Modify the stones
The original Tetris shapes are in a list of arrays called tetris_shapes. The numbers signify the color to use (in the textures). First change the base shapes by making sure your shapes are square (equal number of columns in every row) then you simply put a number where the shape will contain material and leave a 0 where the shape will not contain material. Can you make some shapes that makes gameplay very easy? What kind of shapes are these? On the other hand can you make shapes that making the game so challenging you may clear any rows?

your markdown answer here for [Exercise 1c]

### [Exercise 1d] Modify the number of stones
Now you will add two more stones to either the original tetris values or your modified values in [Exercise 1c]. You will need to make sure you construct a color for each of these new stones by adding a RGBA (Red, Green, Blue, Alpha) to the variable called colors and then will need to make the stones have 8s and 9s like:
[[8,8,8],[8,8,0],[8,8,8]], [[9],[9],[9]]

your markdown here for [exercise 1d]

### [Part 2 20pts] Change the rules to make the game easier
Now that you have a good feel for how to change the difficulty level by modifications to the animation speed, dimensions of the board, and by constructing new stone shapes you will experiment with modifications to the game rules. Normally you need clear a row only by filling it fully. Experiment with clearing rows that have up to one empty space and then clearing rows that have up to two empty spaces. You can copy your code modification in the cell below. Some hints will follow to make this more approachable if you haven't spent enough time in Part 1.

Hint1: What part of the code controls the row clearing rule? Look for the method that has this comment: "...Check for rows we can remove..."
Hint2: Now that you know what method clears a row, modify its Boolean condition so that instead of only clearing full rows, it will trigger wen the rows have up to one empty space (this means they will also clear the normal full rows)


In [1]:
#rewrite the method that controls the row clearing here with your modifications so that it will clear rows with up to one empty space

### [Part 3 20pts]* Change the rules to make the game harder
If you located the row clearing function in [Part 2] revisit that game logic and make modifications so that only two consecutive rows can be cleared. You may need to think about this for some time. What would happen is that even if you had one perfect row, you would need a second one to be perfect to also get cleared.

In [ ]:
#rewrite the method that controls the row clearing here with your modifications that require two full consectutive rows before clearing

### [Part 4 20pts]* Now combine [Part 2] and [Part 3]
Once you have that working, you can add your modification from Part 2 to combine them: You need to have at least two consecutive rows that have at most one empty space each to clear.

In [ ]:
#rewrite the method that controls the row clearing here to have the comination of hard and easy row clearing: 
#two consecutive rows that each are mostly full (each one can have at most one empty space)

### [Part 5 20pts]* Difficultly lies with certain shapes
Do the default shapes pose the same difficulty? Currently the stones are selected with equal chance in the new_stone method by this line:

In [ ]:
self.stone = random.choice(tetris_shapes)

There is a very easy to use numpy version of random choice that will allow you as the programmer to set the chance of each stone. Think about this for a minute in terms of game difficulty... if certain stone shapes are known to be more difficult and you as the programmer can change the chances of these, then you can change the difficulty of the game whenever you want!

In [ ]:
np.random.choice([1,2,3],1,p=[0.1,0.8,0.1]) #1 and 3 have a 1/10 chance each, while 2 has a 4/5 chance

Modify the stone shapes so that easier shapes appear as game proceeds closer to a game over. Think about this for a little bit... how can you figure how the how close the game is to being over?

Hint1: A simple way to calculate how close to failure is to compute the number of filled positions in the board (proportion of filled). You could add a method to your class that could calculate how full the board is (0.0 to 1 value)

Hint2: As the proportion of filled increases, change the value of the p in np.random.choice so that you provide easier shapes.

In [ ]:
#write a proportion_filled method here that computes the number of filled positions

In [ ]:
#write your modifications to the new_stone method here that makes the stone shapes easier as the palyer gets closer to failure